In [4]:
# System libraries:
%load_ext autoreload
import gensim, logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
import pandas as pd
import json
import nltk
import numpy as np
import os
import sys
from IPython.core.interactiveshell import InteractiveShell
from pymystem3 import Mystem
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
from collections import Counter
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from datetime import datetime
import re
import gensim.downloader as api
from gensim.models import TfidfModel
from gensim.corpora import Dictionary
InteractiveShell.ast_node_interactivity = "all"

In [5]:
integrum_local_lemmas = pd.read_csv('data/processed/integrum_local_lemmas.csv', converters = {'lemmas_content' : eval, 'lemmas_headline':eval})
integrum_federal_lemmas = pd.read_csv('data/processed/integrum_federal_lemmas.csv', converters = {'lemmas_content' : eval, 'lemmas_headline':eval})

In [6]:
sentences = integrum_local_lemmas.lemmas_content
# train word2vec on the two sentences
model = gensim.models.Word2Vec(sentences, min_count=1)

2018-05-29 13:18:55,674 : INFO : collecting all words and their counts
2018-05-29 13:18:55,677 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-05-29 13:18:56,890 : INFO : PROGRESS: at sentence #10000, processed 3849812 words, keeping 95908 word types
2018-05-29 13:18:58,555 : INFO : PROGRESS: at sentence #20000, processed 8794850 words, keeping 138728 word types
2018-05-29 13:19:00,097 : INFO : PROGRESS: at sentence #30000, processed 13515149 words, keeping 177293 word types
2018-05-29 13:19:01,315 : INFO : collected 201515 word types from a corpus of 16945329 raw words and 36017 sentences
2018-05-29 13:19:01,316 : INFO : Loading a fresh vocabulary
2018-05-29 13:19:04,089 : INFO : min_count=1 retains 201515 unique words (100% of original 201515, drops 0)
2018-05-29 13:19:04,090 : INFO : min_count=1 leaves 16945329 word corpus (100% of original 16945329, drops 0)
2018-05-29 13:19:04,740 : INFO : deleting the raw counts dictionary of 201515 items
2018-05-2

2018-05-29 13:20:02,520 : INFO : EPOCH 3 - PROGRESS: at 83.42% examples, 846625 words/s, in_qsize 5, out_qsize 0
2018-05-29 13:20:03,523 : INFO : EPOCH 3 - PROGRESS: at 87.57% examples, 842384 words/s, in_qsize 5, out_qsize 0
2018-05-29 13:20:04,533 : INFO : EPOCH 3 - PROGRESS: at 92.00% examples, 842434 words/s, in_qsize 5, out_qsize 0
2018-05-29 13:20:05,562 : INFO : EPOCH 3 - PROGRESS: at 95.91% examples, 838682 words/s, in_qsize 5, out_qsize 0
2018-05-29 13:20:06,571 : INFO : EPOCH 3 - PROGRESS: at 99.55% examples, 835330 words/s, in_qsize 5, out_qsize 0
2018-05-29 13:20:06,689 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-05-29 13:20:06,703 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-05-29 13:20:06,708 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-05-29 13:20:06,709 : INFO : EPOCH - 3 : training on 16945329 raw words (16156975 effective words) took 19.3s, 835195 effective words/s
2018-05-29 13:20:07,72

In [39]:
model.most_similar(positive=['турку'],topn=25)

/work/cloud-user/russia_finland/finland_russia/venv/lib/python3.5/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('тампере', 0.8758280873298645),
 ('котка', 0.8636119365692139),
 ('лаппеенрант', 0.834760844707489),
 ('ювяскюль', 0.8180301189422607),
 ('миккель', 0.8075802326202393),
 ('йоэнсуу', 0.798075795173645),
 ('савонлинн', 0.7975867986679077),
 ('лаппенрант', 0.7900992631912231),
 ('хамин', 0.7882909774780273),
 ('оул', 0.7834081649780273),
 ('куопио', 0.7791638374328613),
 ('рованиеми', 0.7790059447288513),
 ('кухмо', 0.7758827805519104),
 ('каяни', 0.772032618522644),
 ('иматра', 0.7624096870422363),
 ('лахти', 0.7536728382110596),
 ('хельсинки', 0.7462298274040222),
 ('коувол', 0.7360109090805054),
 ('эспоо', 0.7307415008544922),
 ('вантаа', 0.7182021141052246),
 ('тромс', 0.716339647769928),
 ('таллин', 0.6946687698364258),
 ('ваас', 0.683160126209259),
 ('стокгольм', 0.6765294671058655),
 ('тарту', 0.662057638168335)]

In [22]:
model.similarity('петрозаводск', 'хельсинки')

/work/cloud-user/russia_finland/finland_russia/venv/lib/python3.5/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  """Entry point for launching an IPython kernel.


0.592449603625547

In [36]:
model.most_similar(positive=['москва'], negative=['политика'], topn=5)

/work/cloud-user/russia_finland/finland_russia/venv/lib/python3.5/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('новосибирск', 0.6010462045669556),
 ('казань', 0.5952593684196472),
 ('пермь', 0.593327522277832),
 ('екатеринбург', 0.5920868515968323),
 ('мурманск', 0.590484619140625)]

In [9]:
sentences_federal = integrum_federal_lemmas.lemmas_content
# train word2vec on the two sentences
model_federal = gensim.models.Word2Vec(sentences_federal, min_count=1)

2018-05-29 13:23:25,839 : INFO : collecting all words and their counts
2018-05-29 13:23:25,842 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-05-29 13:23:27,272 : INFO : PROGRESS: at sentence #10000, processed 6383312 words, keeping 111386 word types
2018-05-29 13:23:29,016 : INFO : PROGRESS: at sentence #20000, processed 12089164 words, keeping 178086 word types
2018-05-29 13:23:30,125 : INFO : collected 204593 word types from a corpus of 16337994 raw words and 25374 sentences
2018-05-29 13:23:30,126 : INFO : Loading a fresh vocabulary
2018-05-29 13:23:33,086 : INFO : min_count=1 retains 204593 unique words (100% of original 204593, drops 0)
2018-05-29 13:23:33,087 : INFO : min_count=1 leaves 16337994 word corpus (100% of original 16337994, drops 0)
2018-05-29 13:23:33,903 : INFO : deleting the raw counts dictionary of 204593 items
2018-05-29 13:23:33,921 : INFO : sample=0.001 downsamples 18 most-common words
2018-05-29 13:23:33,922 : INFO : downsampli

2018-05-29 13:24:30,768 : INFO : EPOCH 4 - PROGRESS: at 46.91% examples, 912335 words/s, in_qsize 5, out_qsize 0
2018-05-29 13:24:31,785 : INFO : EPOCH 4 - PROGRESS: at 56.12% examples, 909954 words/s, in_qsize 5, out_qsize 0
2018-05-29 13:24:32,787 : INFO : EPOCH 4 - PROGRESS: at 62.96% examples, 909428 words/s, in_qsize 5, out_qsize 0
2018-05-29 13:24:33,810 : INFO : EPOCH 4 - PROGRESS: at 69.79% examples, 911377 words/s, in_qsize 5, out_qsize 0
2018-05-29 13:24:34,821 : INFO : EPOCH 4 - PROGRESS: at 74.55% examples, 906976 words/s, in_qsize 5, out_qsize 0
2018-05-29 13:24:35,821 : INFO : EPOCH 4 - PROGRESS: at 78.73% examples, 906761 words/s, in_qsize 5, out_qsize 0
2018-05-29 13:24:36,831 : INFO : EPOCH 4 - PROGRESS: at 83.05% examples, 903648 words/s, in_qsize 6, out_qsize 1
2018-05-29 13:24:37,832 : INFO : EPOCH 4 - PROGRESS: at 88.38% examples, 903475 words/s, in_qsize 5, out_qsize 0
2018-05-29 13:24:38,832 : INFO : EPOCH 4 - PROGRESS: at 94.53% examples, 900407 words/s, in_qsiz

In [30]:
model_federal.most_similar(positive=['спорт'],topn=5)

/work/cloud-user/russia_finland/finland_russia/venv/lib/python3.5/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('спортивный', 0.5988843441009521),
 ('хоккей', 0.5937655568122864),
 ('исканиус', 0.5813450813293457),
 ('катание', 0.5684407949447632),
 ('футбол', 0.5468862056732178)]

In [23]:
model_federal.similarity('петрозаводск', 'хельсинки')

/work/cloud-user/russia_finland/finland_russia/venv/lib/python3.5/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  """Entry point for launching an IPython kernel.


0.4627681215352176